# Transfer selected files

This script allows to copy files spread out across multiple sub folders to a single directory.
Absolute filepaths of select files should be collected in a single column of an excel file.

### *Create mock files for testing*
<b> The function below only needs to run to create mock files! </b>

In [10]:
def create_mockfiles():
    df = pd.read_excel('..//Testfiles//Book1.xlsx')
    for filepath in df['Filepath']:
        splitpath = filepath.split("/")
        splitpath_nofile = splitpath[0:(len(splitpath)-1)]
        path = "/".join(splitpath_nofile)
        if os.path.exists(path) == False:
            os.makedirs(path)
    for filepath in df['Filepath']:
        try:
            with open(filepath, 'w') as f:
                f.write('Create a new text file!')
        except FileNotFoundError:
            print(filepath, "does not exist")

## <i> CODE </i>

In [2]:
import pandas as pd
import os
import shutil
import time


Define transfer parameters <br>
<b> CHANGE THIS TO ADAPT IN- AND OUTPUT! </b>

In [11]:
test = False
while (test == False):
    excel_metadata_file = input('Enter a file path to Excel list: ') 
    print(excel_metadata_file)
    column_name = input('Column in Excel that contains original filepath: ') or 'path'
    print(column_name)
    sep_used = input('file separator used in the excel file: ') or '\\'
    print(sep_used)
    destination = input('Enter a destination folder: ')
    print(destination)
    overwrite = input('Overwrite files in destination folder? (True, False): ') or False
    print(overwrite)
    test = os.path.exists(excel_metadata_file) and (column_name in pd.read_excel(excel_metadata_file).columns) and (os.path.exists(destination))
    if (test == False):
        print('one of your entries was invalid')
        test = input('repeat? ') or True
        if (test == True):
            print('stopped')
    else:
        print('good to go')    

Enter a file path to Excel list:  ..//Testfiles//Book1.xlsx


..//Testfiles//Book1.xlsx


Column in Excel that contains original filepath:  path


path


file separator used in the excel file:  


\


Enter a destination folder:  ..//Output


..//Output


Overwrite files in destination folder? (True, False):  T


T
good to go


## Check transfer success
Create lists to collect information on transfer success <br>
as well as a clearing function to allow lists to be cleard after each run

In [12]:
missingfiles = list()
pre_existing_destfiles = list()
overwrittenfiles =list()
transferredfiles = list()

def clear_lists(mf = missingfiles, 
                pe = pre_existing_destfiles,
                of = overwrittenfiles,
                tf = transferredfiles):
    mf *= 0
    pe *= 0
    of *= 0
    tf *= 0

## Copy Function
The function below is the actual copy function.<br>
Along with copying the files, it also generates a transfer report in the end to check, <br>
which files were not copied due to error.

<b>Parameters:</b>
<br><span style="color:blue"><b>file</b></span>: excel file with column containing file paths
<br><span style="color:blue"><b>col</b></span>: column in <span style="color:blue"><b>file</b></span> containing file paths
<br><span style="color:blue"><b>dest</b></span>: destination to copy files to
<br><span style="color:blue"><b>sep_used</b></span>: file separator used in excel file
<br><span style="color:blue"><b>overwrite</b></span>: overwrite pre-existing files in <span style="color:blue"><b>dest</b></span> (True/False)
<br><span style="color:blue"><b>missingls, dest_existing, overwritten, transferred</b></span>: lists used to generate a transfer report




In [16]:
def collect_copy(file = excel_metadata_file, 
                 col = column_name, 
                 dest = destination,
                 sep_used = sep_used, 
                 overwrite = overwrite, 
                 missingls = missingfiles, 
                 dest_existing = pre_existing_destfiles, 
                 overwritten = overwrittenfiles, 
                 transferred = transferredfiles):
        
    df = pd.read_excel(file)
    for filepath in df[col]:
        filename = os.path.basename(filepath)
        #Check if input file exists
        if os.path.exists(filepath)== False:
            missingls.append(filepath)
            continue
        destfile = os.path.join(destination, os.path.basename(filepath))
        
        # check if output file exists
        if os.path.exists(destfile) == True:
            dest_existing.append(destfile)
            if overwrite == False:
                continue
            else:
                overwritten.append(destfile)
        #transfer
        shutil.copyfile(filepath, destfile)
        transferred.append(filename)
    
    transfer_report = str("".join([dest, 'transfer_report_', time.strftime("%Y%m%d%H%M"), '.txt']))
    line = "\n \n ----------------------------------------------- \n"
    with open(transfer_report, 'w') as fp:
        fp.write("Transfer Report: \n \n ")
        fp.write(line)
        fp.write("Parameters used:\n")
        fp.write("".join(["Metadata file with input: ", file, "\n"]))
        fp.write("".join(["Column with filepath: ", col, "\n"]))
        fp.write("".join(["Destination Folder: ", dest, "\n"]))
        fp.write("".join(["File separator in excel: ", sep_used, "\n"]))
        fp.write("".join(["Overwrite existing files: ", str(overwrite), "\n"]))
        fp.write(line)
        fp.write("".join([ "finished: ", time.strftime("%Y-%m-%d"), ", ",
                 time.strftime("%H:%M:%S") ]))
        fp.write(line)
        fp.write(line)
        fp.write("Missing Input Files: \n\n")
        for item in missingls:
            # write each item on a new line
            fp.write("%s\n" % item)
        fp.write(line)
        fp.write("Files Already Existed in Destination: \n\n")
        for item in dest_existing:
            # write each item on a new line
            fp.write("%s\n" % item)
        fp.write(line)
        fp.write("Destination Files Overwritten: \n\n")
        for item in overwritten:
            # write each item on a new line
            fp.write("%s\n" % item)
        fp.write(line)
        fp.write("Successfully Transfered: \n\n")
        for item in transferred:
            # write each item on a new line
            fp.write("%s\n" % item)
    print("Transfer finished ", time.strftime("%Y%m%d_%H%M%S"))

Run transfer function

In [17]:
clear_lists()    
collect_copy()

Transfer finished  20221004_090312
